In [94]:
import pandas as pd
from zipfile import ZipFile
import os
import math
import numpy as np
from slugify import slugify
# for printing dfs
pd.options.display.max_rows = 100
# for printing lists
pd.options.display.max_seq_items = 50

In [59]:
# Set relative filepaths
# Missouri voter data is obtained via public records request to the Elections Division, Office of Secretary of State
# More info about data source can be found in the README

__file__ = 'os.path.abspath('')'

script_dir = os.path.dirname(__file__)
rel_path = './data/Missouri'
abs_file_path = os.path.join(script_dir, rel_path)

In [60]:
# Show the name of the zipfile that is opened in the next step

files = os.listdir(abs_file_path)

In [61]:
# Read the zipfile

voters = (files[1])
zf = ZipFile(abs_file_path + "/" + voters)

In [62]:
# List files in zipfile

zf.namelist()

['data\\PSR_VotersList_01032023_9-51-24 AM.txt']

In [63]:
# Load data into dataframe, first with no header for processing reasons

voters = pd.read_csv(zf.open('data\\PSR_VotersList_01032023_9-51-24 AM.txt'), sep='\t', header=None)

In [64]:
# Now set the first row as header

voters.columns = voters.iloc[0] 

In [65]:
# Checking the df

voters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4268188 entries, 0 to 4268187
Data columns (total 54 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   County                     object
 1   Voter ID                   object
 2   First Name                 object
 3   Middle Name                object
 4   Last Name                  object
 5   Suffix                     object
 6   House Number               object
 7   House Suffix               object
 8   Pre Direction              object
 9   Street Name                object
 10  Street Type                object
 11  Post Direction             object
 12  Unit Type                  object
 13  Unit Number                object
 14  Non Standard Address       object
 15  Residential City           object
 16  Residential State          object
 17  Residential ZipCode        object
 18  Mailing Address            object
 19  Mailing City               object
 20  Mailing State           

In [66]:
# Slugifying columns

voters.columns = voters.columns.str.replace(r'\W+', '_', regex=True)
voters.columns = [x.lower() for x in voters.columns]

In [67]:
voters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4268188 entries, 0 to 4268187
Data columns (total 54 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   county                     object
 1   voter_id                   object
 2   first_name                 object
 3   middle_name                object
 4   last_name                  object
 5   suffix                     object
 6   house_number               object
 7   house_suffix               object
 8   pre_direction              object
 9   street_name                object
 10  street_type                object
 11  post_direction             object
 12  unit_type                  object
 13  unit_number                object
 14  non_standard_address       object
 15  residential_city           object
 16  residential_state          object
 17  residential_zipcode        object
 18  mailing_address            object
 19  mailing_city               object
 20  mailing_state           

In [68]:
# There are 20 voter history columns. We'll keep the most recent and store the others in a different dataframe.
# Get names of column headers so we can use them to put the cols we need to in another df 

column_headers = list(voters.columns.values)
del_cols = column_headers[35:]
print(del_cols)

['voter_history_2', 'voter_history_3', 'voter_history_4', 'voter_history_5', 'voter_history_6', 'voter_history_7', 'voter_history_8', 'voter_history_9', 'voter_history_10', 'voter_history_11', 'voter_history_12', 'voter_history_13', 'voter_history_14', 'voter_history_15', 'voter_history_16', 'voter_history_17', 'voter_history_18', 'voter_history_19', 'voter_history_20']


In [69]:
# Put voter history 2-20 into a new, separate dataframe

voters2 = pd.DataFrame()
voters2 = pd.concat([voters2,voters[del_cols]],axis=0)

In [70]:
# Checking voter history dataframe

voters2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4268188 entries, 0 to 4268187
Data columns (total 19 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   voter_history_2   object
 1   voter_history_3   object
 2   voter_history_4   object
 3   voter_history_5   object
 4   voter_history_6   object
 5   voter_history_7   object
 6   voter_history_8   object
 7   voter_history_9   object
 8   voter_history_10  object
 9   voter_history_11  object
 10  voter_history_12  object
 11  voter_history_13  object
 12  voter_history_14  object
 13  voter_history_15  object
 14  voter_history_16  object
 15  voter_history_17  object
 16  voter_history_18  object
 17  voter_history_19  object
 18  voter_history_20  object
dtypes: object(19)
memory usage: 618.7+ MB


In [71]:
# Later on we will drop the extended voter history from the original, first df
# But for now we'll keep it, for the sake of the next step
# Check this data against previous MO voter data, and keep only voters not found in old data
# Loading 2020 data

voters20 = pd.read_csv('./data/mo_voters_2020.csv')

In [72]:
# Slugifying columns in 2020 data

voters20.columns = voters20.columns.str.replace(r'\W+', '_', regex=True)
voters20.columns = [x.lower() for x in voters20.columns]
voters20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4609735 entries, 0 to 4609734
Data columns (total 36 columns):
 #   Column                Dtype  
---  ------                -----  
 0   county                object 
 1   voter_id              int64  
 2   first_name            object 
 3   middle_name           object 
 4   last_name             object 
 5   suffix                object 
 6   house_number          float64
 7   house_suffix          object 
 8   pre_direction         object 
 9   street_name           object 
 10  street_type           object 
 11  post_direction        object 
 12  unit_type             object 
 13  unit_number           object 
 14  non_standard_address  object 
 15  city                  object 
 16  state                 object 
 17  zip                   object 
 18  birth_date            object 
 19  reg_date              object 
 20  precinct              object 
 21  precinct_name         object 
 22  split                 object 
 23  townshi

In [98]:
# Comparing voter ID columns
# In 2020, the TAP team received a similar file. We are going to keep any
# registered voters not found in the current file.

idx1 = pd.Index(voters.voter_id)
idx2 = pd.Index(voters20.voter_id)

diff = idx2.difference(idx1).values
print("There are " + str(len(diff)) +  " voters in the 2020 data who are not in the current data.")

There are 706371 voters in the 2020 data who are not in the current data.


In [99]:
# Convert diff array to list

diff = list(diff)

In [101]:
# Put those voters from 2020 data not in current data into a df, we'll need it later

keepers = voters20[voters20['voter_id'].isin(diff)]
keepers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 706372 entries, 0 to 4609734
Data columns (total 36 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   county                706372 non-null  object 
 1   voter_id              706372 non-null  int64  
 2   first_name            706358 non-null  object 
 3   middle_name           641694 non-null  object 
 4   last_name             706316 non-null  object 
 5   suffix                29167 non-null   object 
 6   house_number          671155 non-null  float64
 7   house_suffix          3461 non-null    object 
 8   pre_direction         213309 non-null  object 
 9   street_name           671150 non-null  object 
 10  street_type           598529 non-null  object 
 11  post_direction        5338 non-null    object 
 12  unit_type             113346 non-null  object 
 13  unit_number           113343 non-null  object 
 14  non_standard_address  35534 non-null   object 
 15 

In [13]:
# Drop voter history 2-20 from original dataframe

voters.drop(columns=['voter_history_2', 'voter_history_3', 'voter_history_4', 'voter_history_5', 'voter_history_6', 'voter_history_7', 'voter_history_8', 'voter_history_9', 'voter_history_10', 'voter_history_11', 'voter_history_12', 'voter_history_13', 'voter_history_14', 'voter_history_15', 'voter_history_16', 'voter_history_17', 'voter_history_18', 'voter_history_19', 'voter_history_20'], inplace=True)

In [14]:
voters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4268188 entries, 0 to 4268187
Data columns (total 35 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   County                     object
 1   Voter ID                   object
 2   First Name                 object
 3   Middle Name                object
 4   Last Name                  object
 5   Suffix                     object
 6   House Number               object
 7   House Suffix               object
 8   Pre Direction              object
 9   Street Name                object
 10  Street Type                object
 11  Post Direction             object
 12  Unit Type                  object
 13  Unit Number                object
 14  Non Standard Address       object
 15  Residential City           object
 16  Residential State          object
 17  Residential ZipCode        object
 18  Mailing Address            object
 19  Mailing City               object
 20  Mailing State           

In [17]:
voters20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4609735 entries, 0 to 4609734
Data columns (total 36 columns):
 #   Column                Dtype  
---  ------                -----  
 0   county                object 
 1   voter_id              int64  
 2   first_name            object 
 3   middle_name           object 
 4   last_name             object 
 5   suffix                object 
 6   house_number          float64
 7   house_suffix          object 
 8   pre_direction         object 
 9   street_name           object 
 10  street_type           object 
 11  post_direction        object 
 12  unit_type             object 
 13  unit_number           object 
 14  non_standard_address  object 
 15  city                  object 
 16  state                 object 
 17  zip                   object 
 18  birth_date            object 
 19  reg_date              object 
 20  precinct              object 
 21  precinct_name         object 
 22  split                 object 
 23  townshi

In [ ]:
idx1 = pd.Index(voters.)
idx2 = pd.Index(df.id2)

idx1.difference(idx2).values